### 1. CREAR BASE DE DATOS (33%)

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.

In [13]:
import pandas as pd
import mysql.connector as con
import random
import datetime

In [14]:
# Crear base de datos "supermercado"
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin"
)
cursor = conexion.cursor()
cursor.execute("DROP DATABASE IF EXISTS supermercado;")
cursor.execute("CREATE SCHEMA supermercado DEFAULT CHARACTER SET utf8;")
#cursor.close()
#connection.close()
#connection.close()

In [32]:
# Creo una sola tabla
# Conexion
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

# crear tabla
query_tabla_tiendas = """
CREATE TABLE IF NOT EXISTS tiendas (
    id_tienda INT AUTO_INCREMENT PRIMARY KEY,
    nombre_tienda VARCHAR(30),
    direccion_tienda VARCHAR(50),
    ciudad VARCHAR (50)
)
"""
# Hacer los cambios en la base de datos
cursor.execute(query_tabla_tiendas)
conexion.commit()

# Cerra la conexion
cursor.close()
conexion.close()

In [ ]:
# Creo varias tablas a la vez
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

query_varias_tablas = ["""
    CREATE TABLE IF NOT EXISTS empleados (
    id_empleado INT AUTO_INCREMENT PRIMARY KEY,
    nombre_empleado VARCHAR(30),
    puesto VARCHAR(20),
    FOREING KEY (id_tienda) REFERENCES tiendas(tienda_id) 
)"""














]



for query in query_varias_tablas:
    cursor.execute(query)
    conexion.commit()

print("Todas las tablas han sido creadas exitosamente.")

# Cerrar la conexión
cursor.close()
conexion.close()